# Module 1 - Programming Assignment

## General Directions

1. You must follow the Programming Requirements outlined on Canvas.
2. The Notebook should be cleanly and fully executed before submission.
3. You should change the name of this file to be your JHED id. For example, `jsmith299.ipynb` although Canvas will change it to something else... :/

<div style="background: lemonchiffon; margin:20px; padding: 20px;">
    <strong>Important</strong>
    <p>
        You should always read the entire assignment before beginning your work, so that you know in advance what the requested output will be and can plan your implementation accordingly.
    </p>
</div>

# State Space Search with A* Search

You are going to implement the A\* Search algorithm for navigation problems.


## Motivation

Search is often used for path-finding in video games. Although the characters in a video game often move in continuous spaces,
it is trivial to layout a "waypoint" system as a kind of navigation grid over the continuous space. Then if the character needs
to get from Point A to Point B, it does a line of sight (LOS) scan to find the nearest waypoint (let's call it Waypoint A) and
finds the nearest, LOS waypoint to Point B (let's call it Waypoint B). The agent then does a A* search for Waypoint B from Waypoint A to find the shortest path. The entire path is thus Point A to Waypoint A to Waypoint B to Point B.

We're going to simplify the problem by working in a grid world. The symbols that form the grid have a special meaning as they
specify the type of the terrain and the cost to enter a grid cell with that type of terrain:

```
token   terrain    cost 
🌾       plains     1
🌲       forest     3
🪨       hills      5
🐊       swamp      7
🗻       mountains  impassible
```

We can think of the raw format of the map as being something like:

```
🌾🌾🌾🌾🌲🌾🌾
🌾🌾🌾🌲🌲🌲🌾
🌾🗻🗻🗻🌾🌾🌾
🌾🌾🗻🗻🌾🌾🌾
🌾🌾🗻🌾🌾🌲🌲
🌾🌾🌾🌾🌲🌲🌲
🌾🌾🌾🌾🌾🌾🌾
```

## The World

Given a map like the one above, we can easily represent each row as a `List` and the entire map as `List of Lists`:

In [96]:
full_world = [
['🌾', '🌾', '🌾', '🌾', '🌾', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾', '🌾', '🗻', '🗻', '🗻', '🗻', '🗻', '🗻', '🗻', '🌾', '🌾'],
['🌾', '🌾', '🌾', '🌾', '🗻', '🗻', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🗻', '🗻', '🗻', '🪨', '🪨', '🪨', '🗻', '🗻', '🪨', '🪨'],
['🌾', '🌾', '🌾', '🌾', '🪨', '🗻', '🗻', '🗻', '🌲', '🌲', '🌲', '🌲', '🐊', '🐊', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾', '🌾', '🪨', '🪨', '🗻', '🗻', '🪨', '🌾'],
['🌾', '🌾', '🌾', '🪨', '🪨', '🗻', '🗻', '🌲', '🌲', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '🌲', '🌲', '🌲', '🌾', '🌾', '🌾', '🪨', '🗻', '🗻', '🗻', '🪨', '🌾'],
['🌾', '🪨', '🪨', '🪨', '🗻', '🗻', '🪨', '🪨', '🌾', '🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '🐊', '🌾', '🌾', '🌾', '🌾', '🌾', '🪨', '🗻', '🪨', '🌾', '🌾'],
['🌾', '🪨', '🪨', '🗻', '🗻', '🪨', '🪨', '🌾', '🌾', '🌾', '🌾', '🪨', '🗻', '🗻', '🗻', '🐊', '🐊', '🐊', '🌾', '🌾', '🌾', '🌾', '🌾', '🪨', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🪨', '🪨', '🪨', '🪨', '🪨', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🪨', '🗻', '🗻', '🗻', '🐊', '🐊', '🐊', '🌾', '🌾', '🪨', '🪨', '🪨', '🌾', '🌾'],
['🌾', '🌾', '🌾', '🪨', '🪨', '🪨', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🪨', '🪨', '🗻', '🗻', '🌾', '🐊', '🐊', '🌾', '🌾', '🪨', '🪨', '🪨', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🌾', '🌾', '🪨', '🪨', '🪨', '🗻', '🗻', '🗻', '🗻', '🌾', '🌾', '🌾', '🐊', '🌾', '🪨', '🪨', '🪨', '🌾', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '🐊', '🌾', '🪨', '🪨', '🗻', '🗻', '🗻', '🪨', '🌾', '🌾', '🌾', '🌾', '🌾', '🪨', '🗻', '🗻', '🗻', '🪨', '🌾', '🌾', '🌾'],
['🌾', '🐊', '🐊', '🐊', '🐊', '🐊', '🌾', '🌾', '🪨', '🗻', '🗻', '🪨', '🌾', '🌾', '🌾', '🌾', '🐊', '🐊', '🌾', '🌾', '🪨', '🗻', '🗻', '🪨', '🌾', '🌾', '🌾'],
['🐊', '🐊', '🐊', '🐊', '🐊', '🌾', '🌾', '🪨', '🪨', '🗻', '🗻', '🪨', '🌾', '🐊', '🐊', '🐊', '🐊', '🌾', '🌾', '🌾', '🪨', '🗻', '🪨', '🌾', '🌾', '🌾', '🌾'],
['🌾', '🐊', '🐊', '🐊', '🐊', '🌾', '🌾', '🪨', '🌲', '🌲', '🪨', '🌾', '🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '🌾', '🌾', '🪨', '🌾', '🌾', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🌾', '🌾', '🗻', '🌾', '🌾', '🌲', '🌲', '🌲', '🌲', '🪨', '🪨', '🪨', '🪨', '🌾', '🐊', '🐊', '🐊', '🌾', '🌾', '🪨', '🗻', '🪨', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🌾', '🗻', '🗻', '🗻', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🗻', '🗻', '🗻', '🪨', '🪨', '🌾', '🐊', '🌾', '🪨', '🗻', '🗻', '🪨', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🗻', '🗻', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🗻', '🗻', '🗻', '🌾', '🌾', '🗻', '🗻', '🗻', '🌾', '🌾', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🌾', '🗻', '🗻', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🗻', '🗻', '🗻', '🗻', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🌾', '🗻', '🗻', '🗻', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾', '🌾', '🌾', '🪨', '🪨', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🌾', '🌾', '🗻', '🗻', '🗻', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊'],
['🌾', '🌾', '🪨', '🪨', '🪨', '🪨', '🗻', '🗻', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾', '🗻', '🌾', '🌾', '🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '🐊', '🐊', '🐊'],
['🌾', '🌾', '🌾', '🌾', '🪨', '🪨', '🪨', '🗻', '🗻', '🗻', '🌲', '🌲', '🗻', '🗻', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '🐊', '🐊', '🐊'],
['🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🪨', '🪨', '🪨', '🗻', '🗻', '🗻', '🗻', '🌾', '🌾', '🌾', '🌾', '🪨', '🪨', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '🐊', '🐊'],
['🌾', '🪨', '🪨', '🌾', '🌾', '🪨', '🪨', '🪨', '🪨', '🪨', '🌾', '🌾', '🌾', '🌾', '🌾', '🪨', '🪨', '🗻', '🗻', '🪨', '🪨', '🌾', '🐊', '🐊', '🐊', '🐊', '🐊'],
['🪨', '🗻', '🪨', '🪨', '🪨', '🪨', '🌾', '🌾', '🌾', '🌾', '🌾', '🗻', '🗻', '🗻', '🪨', '🪨', '🗻', '🗻', '🌾', '🗻', '🗻', '🪨', '🪨', '🐊', '🐊', '🐊', '🐊'],
['🪨', '🗻', '🗻', '🗻', '🪨', '🌾', '🌾', '🌾', '🌾', '🌾', '🪨', '🪨', '🗻', '🗻', '🗻', '🗻', '🪨', '🪨', '🪨', '🪨', '🗻', '🗻', '🗻', '🐊', '🐊', '🐊', '🐊'],
['🪨', '🪨', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🪨', '🪨', '🪨', '🪨', '🪨', '🌾', '🌾', '🌾', '🌾', '🪨', '🪨', '🪨', '🌾', '🌾', '🌾']
]

## Warning

One implication of this representation is that (x, y) is world[ y][ x] so that (3, 2) is world[ 2][ 3] and world[ 7][ 9] is (9, 7). Yes, there are many ways to do this. I picked this representation because when you look at it, it *looks* like a regular x, y cartesian grid and it's easy to print out.

It is often easier to begin your programming by operating on test input that has an obvious solution. If we had a small 7x7 world with the following characteristics:

In [97]:
test_world = [
    ['🌾', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲'],
    ['🌾', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲'],
    ['🌾', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲'],
    ['🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾'],
    ['🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾'],
    ['🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾'],
    ['🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾']
]

**what do you expect the policy would be?** Think about it for a bit. This will help you with your programming and debugging.

## States and State Representation

The canonical pieces of a State Space Search problem are the States, Actions, Transitions and Costs. 

We'll start with the state representation. For the navigation problem, a state is the current position of the agent, `(x,y)`. The entire set of possible states is implicitly represented by the world map.

## Actions and Transitions

Next we need to specify the actions. In general, there are a number of different possible action sets in such a world. The agent might be constrained to move north/south/east/west or diagonal moves might be permitted as well (or really anything). When combined with the set of States, the *permissible* actions forms the Transition set.

Rather than enumerate the Transition set directly, for this problem it's easier to calculate the available actions and transitions on the fly. This can be done by specifying a *movement model* as offsets to the current state and then checking to see which of the potential successor states are actually permitted. This can be done in the successor function mentioned in the pseudocode.

One such example of a movement model is shown below.

In [98]:
MOVES = [(0,-1), (1,0), (0,1), (-1,0)]

## Costs

We can encode the costs described above in a `Dict`:

In [99]:
COSTS = { '🌾': 1, '🌲': 3, '🪨': 5, '🐊': 7}

## Specification

You will implement a function called `a_star_search` that takes the parameters and returns the value as specified below. The return value is going to look like this:

`[(0,1), (0,1), (0,1), (1,0), (1,0), (1,0), (1,0), (1,0), (1,0), (0,1), (0,1), (0,1)]`

You should also implement a function called `pretty_print_path`. 
The `pretty_print_path` function prints an ASCII representation of the path generated by the `a_star_search` on top of the terrain map. 
For example, for the test world, it would print this:

```
⏬🌲🌲🌲🌲🌲🌲
⏬🌲🌲🌲🌲🌲🌲
⏬🌲🌲🌲🌲🌲🌲
⏩⏩⏩⏩⏩⏩⏬
🌲🌲🌲🌲🌲🌲⏬
🌲🌲🌲🌲🌲🌲⏬
🌲🌲🌲🌲🌲🌲🎁
```

using ⏩,⏪,⏫ ⏬ to represent actions and `🎁` to represent the goal. (Note the format of the output...there are no spaces, commas, or extraneous characters). You are printing the path over the terrain.
This is an impure function (it does not return anything).

Note that in Python:
```
> a = ["*", "-", "*"]
> "".join(a)
*-*
```
Do not print raw data structures; do not insert unneeded/requested spaces!

### Additional comments

As Python is an interpreted language, you're going to need to insert all of your functions *before* the actual `a_star_search` function implementation. 
Do not make unwarranted assumptions (for example, do not assume that the start is always (0, 0).
Do not refer to global variables, pass them as parameters (functional programming).

Simple and correct is better than inefficient and incorrect, or worse, incomplete.
For example, you can use a simple List, with some helper functions, as a Stack or a Queue or a Priority Queue.
Avoid the Python implementations of HeapQ, PriorityQueue implementation unless you are very sure about what you're doing as they require *immutable* keys.

In [100]:
from typing import List, Tuple, Dict, Callable
from copy import deepcopy

In [101]:
from collections import defaultdict
import queue

*add as many markdown and code cells here as you need for helper functions. We have added `heuristic` for you*

### <a id="path"></a> path

Formal Parameters:
**came_from** A dictionary with key:value pairs location:previous location

**current** The current location

**returns** total_path a list of the locations in order

This is used to convert a dictionary to a list

In [102]:
def path(came_From, current):
    total_Path = [current]
    while current in came_From.keys():
        current = came_From[current]
        total_Path = [current] + total_Path
    return total_Path

In [103]:
came_from1 = {1:2,2:3,3:4}
current1 = 1
assert(path(came_from1,current1) == [4,3,2,1])
current2 = 2
assert(path(came_from1,current2) == [4,3,2])
current3 = 3
assert(path(came_from1,current3) == [4,3])

### <a id="move_List"></a> move_List

Formal Parameters:
**path** a list of the locations in order

**world** The world

**costs** The costs dictionary

**returns** `List,total_Cost` a list of moves required to follow the path, the cost of that path

This is used to calculate the moves and cost required for a path.

In [104]:
def move_List(path,world,costs):
    total_Cost = 0
    List = []
    
    for i in range(len(path)-1):
        current = path[i]
        next_Node=path[i+1]
        total_Cost+= costs[world[next_Node[0]][next_Node[1]]]
        move = (next_Node[1]-current[1],next_Node[0]-current[0])
        List.append(move)
    
    return List,total_Cost

In [105]:
assert(move_List([(0,0),(0,1)],test_world,COSTS)==([(1, 0)], 3))
assert(move_List([(0,0),(1,0)],test_world,COSTS)==([(0, 1)], 1))
assert(move_List([(0,0),(1,0),(2,0)],test_world,COSTS)==([(0, 1),(0,1)], 2))

### <a id="highlighted_Path"></a> highlighted_Path

Formal Parameters:
**move_List** a list of the locations in order

**world** The world

**costs** The costs dictionary

**start** The starting position

**returns** `path,total_Cost` the path, the cost of that path

This is used to calculate the path and cost required for a path.

In [106]:
def highlighted_Path(move_List,world,costs,start):
    current = start
    path = [start]
    total_Cost = 0
    for m in move_List:
        current = (current[0]+m[1],current[1]+m[0])
        path.append(current)
        total_Cost += costs[world[current[0]][current[1]]]
    return path,total_Cost

### <a id="find_Neighbors"></a> find_Neighbors

Formal Parameters:
**node** a grid in the world

**world** The world

**returns** `neighbors` a list containing all adjacent grids to `node` that are not mountains

This is used to in the search algorithm to determine the candidates for the next move

In [107]:
def find_Neighbors(node,world):
    neighbors = []
    if node[0] > 0 and world[node[0]-1][node[1]] != '🗻':
        neighbors.append((node[0]-1,node[1]))
    if node[0] < len(world)-1 and world[node[0]+1][node[1]] != '🗻':
        neighbors.append((node[0]+1,node[1]))
    if node[1] > 0 and world[node[0]][node[1]-1] != '🗻':
        neighbors.append((node[0],node[1]-1))
    if node[1] < len(world[0])-1 and world[node[0]][node[1]+1] != '🗻':
        neighbors.append((node[0],node[1]+1))
    return neighbors

In [108]:
assert(find_Neighbors((0,0),test_world)==[(1, 0), (0, 1)])
assert(find_Neighbors((1,1),test_world)==[(0, 1), (2, 1), (1, 0), (1, 2)])
assert(find_Neighbors((2,1),test_world)==[(1, 1), (3, 1), (2, 0), (2, 2)])

<a id="heuristic"></a>
## heuristic

Formal Parameters:
**start** a starting grid

**end** The hypothetical ending grid

**returns** a heuristic cost based on start and end

It's actually the Manhattan distance

In [109]:
def heuristic(start,end): # you can add formal parameters
    xDiff = abs(start[0] - end[0])
    yDiff = abs(start[1] - end[1])
    return xDiff+yDiff

In [110]:
# assertions/unit tests
assert heuristic((1,2),(3,4)) == 4
assert heuristic((1,6),(3,4)) == 4
assert heuristic((5,2),(3,4)) == 4

<a id="a_star_search"></a>
## a_star_search

*add documentation/description of the algorithm here; connect it to the theory!*

* **world** List[List[str]]: the actual context for the navigation problem.
* **start** Tuple[int, int]: the starting location of the bot, `(x, y)`.
* **goal** Tuple[int, int]: the desired goal position for the bot, `(x, y)`.
* **costs** Dict[str, int]: is a `Dict` of costs for each type of terrain in **world**.
* **moves** List[Tuple[int, int]]: the legal movement model expressed in offsets in **world**.
* **heuristic** Callable: is a heuristic function, $h(n)$.


**returns** List[Tuple[int, int]]: the offsets needed to get from start state to the goal as a `List`.





<h1><a id="a_star_search-ALGORITHM" class="anchor" href="#ALGORITHM">A* Search Algorithm</a></h1>
#### Formal Paramaters: *world*: The microworld, *start*: The starting point, *goal*, The goal point, *costs*, A map from terrain type
#### to cost, *moves*: The list of possible moves, *heuristic*: The heuristic function to estimate cost.
#### Returns **A list of moves that will get from *start* to *goal* optimally**
###### A star search is a best first search algorithm that uses a heuristic function to find the optimal path more quickly.

In [111]:
def a_star_search( world: List[List[str]], start: Tuple[int, int], goal: Tuple[int, int], costs: Dict[str, int], moves: List[Tuple[int, int]], heuristic: Callable) -> List[Tuple[int, int]]:
    frontier,frontier_List,came_From,g,f = queue.PriorityQueue(), [start],{}, defaultdict(lambda:float('inf')), defaultdict(lambda:float('inf'))
    frontier.put((0,start))
    g[start]=0
    f[start]=heuristic(start,goal)
    while not frontier.empty():
        current = frontier.get()[1]
        if current == goal:
            return move_List(path(came_From,current),world,costs)[0]
        frontier_List.remove(current)
        for n in find_Neighbors(current,world):
            temp_G = g[current]+costs[world[n[0]][n[1]]]
            if temp_G < g[n]:
                came_From[n] = current
                g[n]=temp_G
                f[n]=temp_G+heuristic(start,n)
                if n not in frontier_List:
                    frontier_List.append(n)
                    frontier.put((f[n],n))
    return [] 

## pretty_print_path

*write your documentation here*

* **world** List[List[str]]: the world (terrain map) for the path to be printed upon.
* **path** List[Tuple[int, int]]: the path from start to goal, in offsets.
* **start** Tuple[int, int]: the starting location for the path.
* **goal** Tuple[int, int]: the goal location for the path.

**returns** int - The path cost.


<div style="background: lemonchiffon; margin:20px; padding: 20px;">
    <strong>Important</strong>
    <p>
        Does your output of pretty_print_path really look like the specification? Go check again.
    </p>
</div>

In [112]:
def pretty_print_solution( world: List[List[str]], path: List[Tuple[int, int]], start: Tuple[int, int], goal: Tuple[int, int]) -> int:
    ### YOUR SOLUTION HERE ###
    ### YOUR SOLUTION HERE ###
    action_Dict = {(1,0):'⏩',(-1,0):'⏪',(0,1):'⏬',(0,-1):'⏫'}
    path_List,total_Cost = highlighted_Path(path,world,COSTS,start)
    print_World = deepcopy(world)
    for i in range(len(path_List)-1):
        print_World[path_List[i][0]][path_List[i][1]] = action_Dict[path[i]]
        
    print_World[goal[0]][goal[1]]='🎁'
    for line in print_World:
        print("".join(line))
    #print(printWorld)
    
    return total_Cost

## Problems

## Problem 1. 

Execute `a_star_search` and `print_path` for the `test_world`.

If you change any values while developing your code, make sure you change them back! (Better yet, don't do it. Copy them elsewhere and change the values, then delete those experiments).

In [113]:
test_start = (0, 0)
test_goal = (len(test_world[0]) - 1, len(test_world) - 1)
test_path = a_star_search(test_world, test_start, test_goal, COSTS, MOVES, heuristic)
test_path_cost = pretty_print_solution(test_world, test_path, test_start, test_goal)
print(f"total path cost: {test_path_cost}")
print(test_path)

⏬🌲🌲🌲🌲🌲🌲
⏬🌲🌲🌲🌲🌲🌲
⏬🌲🌲🌲🌲🌲🌲
⏩⏩⏩⏩⏩⏩⏬
🌲🌲🌲🌲🌲🌲⏬
🌲🌲🌲🌲🌲🌲⏬
🌲🌲🌲🌲🌲🌲🎁
total path cost: 12
[(0, 1), (0, 1), (0, 1), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (0, 1), (0, 1), (0, 1)]


## Problem 2

Execute `a_star_search` and `print_path` for the `full_world`

In [114]:
full_start = (0, 0)
full_goal = (len(full_world[0]) - 1, len(full_world) - 1)
full_path = a_star_search(full_world, full_start, full_goal, COSTS, MOVES, heuristic)
full_path_cost = pretty_print_solution(full_world, full_path, full_start, full_goal)
print(f"total path cost: {full_path_cost}")
print(full_path)

⏬🌾🌾🌾🌾🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌾🌾🌾🌾🌾🌾🌾🌾🌾🌾🌾🌾
⏬🌾🌾🌾🌾🌾🌾🌲🌲🌲🌲🌲🌲🌲🌲🌲🌾🌾🗻🗻🗻🗻🗻🗻🗻🌾🌾
⏬🌾🌾🌾🗻🗻🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🗻🗻🗻🪨🪨🪨🗻🗻🪨🪨
⏬🌾🌾🌾🪨🗻🗻🗻🌲🌲🌲🌲🐊🐊🌲🌲🌲🌲🌲🌾🌾🪨🪨🗻🗻🪨🌾
⏬🌾🌾🪨🪨🗻🗻🌲🌲🌾🌾🐊🐊🐊🐊🌲🌲🌲🌾🌾🌾🪨🗻🗻🗻🪨🌾
⏬🪨🪨🪨🗻🗻🪨🪨🌾🌾🌾🌾🐊🐊🐊🐊🐊🌾🌾🌾🌾🌾🪨🗻🪨🌾🌾
⏬🪨🪨🗻🗻🪨🪨🌾🌾🌾🌾🪨🗻🗻🗻🐊🐊🐊🌾🌾🌾🌾🌾🪨🌾🌾🌾
⏬🌾🪨🪨🪨🪨🪨🌾🌾🌾🌾🌾🌾🪨🗻🗻🗻🐊🐊🐊🌾🌾🪨🪨🪨🌾🌾
⏬🌾🌾🪨🪨🪨🌾🌾🌾🌾🌾🌾🪨🪨🗻🗻🌾🐊🐊🌾🌾🪨🪨🪨🌾🌾🌾
⏬🌾🌾🐊🐊🐊🌾🌾🪨🪨🪨🗻🗻🗻🗻🌾🌾🌾🐊🌾🪨🪨🪨🌾🌾🌾🌾
⏬🌾🐊🐊🐊🐊🐊🌾🪨🪨🗻🗻🗻🪨🌾🌾🌾🌾🌾🪨🗻🗻🗻🪨🌾🌾🌾
⏬🐊🐊🐊🐊🐊🌾🌾🪨🗻🗻🪨🌾🌾🌾🌾🐊🐊🌾🌾🪨🗻🗻🪨🌾🌾🌾
⏬🐊🐊🐊🐊🌾🌾🪨🪨🗻🗻🪨🌾🐊🐊🐊🐊🌾🌾🌾🪨🗻🪨🌾🌾🌾🌾
⏬🐊🐊🐊🐊🌾🌾🪨🌲🌲🪨🌾🌾🌾🌾🐊🐊🐊🐊🌾🌾🪨🌾🌾🌾🌾🌾
⏩⏬🌾🌾🗻🌾🌾🌲🌲🌲🌲🪨🪨🪨🪨🌾🐊🐊🐊🌾🌾🪨🗻🪨🌾🌾🌾
🌾⏬🌾🗻🗻🗻🌲🌲🌲🌲🌲🌲🗻🗻🗻🪨🪨🌾🐊🌾🪨🗻🗻🪨🌾🌾🌾
🌾⏬🗻🗻🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🗻🗻🗻🌾🌾🗻🗻🗻🌾🌾🌾🌾🌾
🌾⏬🌾🗻🗻🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🗻🗻🗻🗻🌾🌾🌾🌾🌾🌾🌾
🌾⏬🌾🗻🗻🗻🌲🌲🌲🌲🌲🌲🌲🌲🌾🌾🌾🪨🪨🌾🌾🌾🌾🌾🌾🌾🌾
🌾⏬🌾🌾🗻🗻🗻🌲🌲🌲🌲🌲🌲🌾🌾🌾🌾🌾🌾🌾🌾🌾🌾🐊🐊🐊🐊
🌾⏬🪨🪨🪨🪨🗻🗻🌲🌲🌲🌲🌲🌾🗻🌾🌾🌾🌾🌾🐊🐊🐊🐊🐊🐊🐊
🌾⏩⏩⏬🪨🪨🪨🗻🗻🗻🌲🌲🗻🗻🌾🌾🌾🌾🌾🌾🐊🐊🐊🐊🐊🐊🐊
🌾🌾🌾⏩⏩⏩⏬🪨🪨🗻🗻🗻🗻🌾🌾🌾🌾🪨🪨🌾🌾🐊🐊🐊🐊🐊🐊
🌾🪨🪨🌾🌾🪨⏬🪨🪨🪨🌾🌾🌾🌾🌾🪨🪨🗻🗻🪨🪨🌾🐊🐊🐊🐊🐊
🪨🗻🪨🪨🪨🪨⏩⏩⏩⏬🌾🗻🗻🗻🪨🪨🗻🗻🌾🗻🗻🪨🪨🐊🐊🐊🐊
🪨🗻🗻🗻🪨🌾🌾🌾🌾⏬🪨🪨🗻🗻🗻🗻🪨🪨🪨🪨🗻🗻🗻🐊🐊🐊🐊
🪨🪨🌾🌾🌾🌾🌾🌾🌾⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩🎁
total path cost: 98
[(0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (1, 0), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (1, 0), (1, 0), (0, 1), (1, 0), (1, 0), (1, 0),

## Comments

(This is the place to leave me comments)

In [115]:
#You don't want us to use global variables in the function, but pretty_print doesn't take COSTS as a parameter?
#am I misunderstanding the directions?

## To think about for future assignments...

This first assignment may not have been difficult for you if you've encountered A* search before in your Algorithms course. In preparation for future assignments that build on State Space Search, you can think about the following or even do an implementation if you like. You should **not** submit it as part of this assignment.

In several future assignments, we will have a need for a "plain ol'" Depth First Search algorithm.

1. Implement DFS Search to solve the problem presented in this programming assignment. Try to be as general as possible (don't hard code anything you can pass as a formal parameter).
2. Can you implement DFS Search as a higher order function and supply your own `is_goal`, `successors`, and `path` functions? How do you handle *state*?
3. Can you write a version of DFS that returns all the solutions?

In one future assignment a Breadth First Search algorithm will be very handy. Can you implement a search algorithm that changes whether it uses DFS or BFS by parameterization?

## Before You Submit...

1. Did you provide output exactly as requested?
2. Did you follow the Programming Requirements on Canvas?

Do not submit any other files.